In [1]:
import urllib.request, json, xmltodict
import numpy as np
import pandas as pd
import re
import gensim
import pymorphy2

C:\Users\dayan\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [145]:
reviews = pd.DataFrame.from_csv("./res.csv", encoding="cp1251")
reviews

,rating,content
0,1,"В описании приложения, написали что есть русск..."
1,1,Пожалуйста сделайте русский язык!!!!!
2,2,"Зашла в игру, поиграла ровно минуту и всё, эне..."
3,2,"Игра интересная, но как же достал connect erro..."
4,5,"Будьте любезны, если есть возможность добавьте..."
5,1,Where is the Russian language? why the descrip...
6,4,Без русского играть невозможно. От выборы зави...
7,5,"Почему игра не запускается, перекидывает на ст..."
8,1,Когда же ждать заявленный в описании русский я...
9,2,"Русского языка нет, но английский понимаю - та..."


In [146]:
pm2_to_univ_POS = {
    "NOUN" : "NOUN",
    "ADJF" : "ADJ",
    "ADJS" : "ADJ",
    "COMP" : "ADJ",
    "VERB" : "VERB",
    "INFN" : "VERB",
    "PRTF" : "ADJ",
    "PRTS" : "ADJ",
    "GRND" : "ADV",
    "NUMR" : "NUM",
    "ADVB" : "ADV",
    "NPRO" : "PRON",
    "PRED" : "ADV",
    "PREP" : "ADP",
    "CONJ" : "CCONJ",
    "PRCL" : "PART",
    "INTJ" : "INTJ"
} 

In [148]:
#drop NaN reviews
reviews.drop(reviews[pd.isnull(reviews.content)].index, inplace=True)
#drop empty reviews
reviews.drop(reviews[reviews.content == ""].index, inplace=True)
#drop eng reviews
reviews.drop(reviews[reviews.content.str.contains ("[A-Za-z]")].index, inplace=True)

reviews

,rating,content
0,1,"В описании приложения, написали что есть русск..."
1,1,Пожалуйста сделайте русский язык!!!!!
2,2,"Зашла в игру, поиграла ровно минуту и всё, эне..."
4,5,"Будьте любезны, если есть возможность добавьте..."
6,4,Без русского играть невозможно. От выборы зави...
7,5,"Почему игра не запускается, перекидывает на ст..."
8,1,Когда же ждать заявленный в описании русский я...
9,2,"Русского языка нет, но английский понимаю - та..."
10,5,"Товарищи, учите иностранные языки \r\nИгра зам..."
12,1,Заявлен русский язык. Заходишь в игре в Настро...


In [149]:
morph = pymorphy2.MorphAnalyzer()

def strtonormalform(line, should_delete = {'PREP', 'CONJ', 'PRCL', 'INTJ'}):
    ans = ''
    words = re.findall(r"[\w]+[\w|-]*", line)
    for word in words:
        lexInfo = morph.parse(word)[0]
        lex = lexInfo.normal_form
        if (lexInfo.tag.POS not in should_delete) and (re.match(r"[-+]?\d+$", lex) is None) :#союзы, предлоги и т.п. не сохраняем
            ans = ans + ' ' + lex
            if (lexInfo.tag.POS):
                ans = ans + "_" + lexInfo.tag.POS
        
    return ans



reviews['content'] = reviews['content'].apply(strtonormalform)

In [150]:
reviews

,rating,content
0,1,описание_NOUN приложение_NOUN написать_VERB р...
1,1,сделать_VERB русский_ADJF язык_NOUN
2,2,зайти_VERB игра_NOUN поиграть_VERB ровно_ADVB...
4,5,быть_VERB любезный_ADJS возможность_NOUN доба...
6,4,русский_ADJF играть_INFN невозможно_ADVB выбо...
7,5,почему_ADVB игра_NOUN запускаться_VERB переки...
8,1,ждать_INFN заявить_PRTF описание_NOUN русский...
9,2,русский_ADJF язык_NOUN нет_PRED английский_AD...
10,5,товарищ_NOUN учить_VERB иностранный_ADJF язык...
12,1,заявить_PRTS русский_ADJF язык_NOUN заходить_...


In [187]:
from sklearn.feature_extraction.text import CountVectorizer

data = reviews.content.values.astype('U')
cnt_vec = CountVectorizer()
transformed_data = cnt_vec.fit_transform(data)

freqs_dict = {"names" : cnt_vec.get_feature_names(), "freqs" : transformed_data.sum(axis=0).tolist()[0]}

word_freqs = pd.DataFrame(freqs_dict)
word_freqs.sort_values(by='freqs', inplace=True, ascending=False)
word_freqs

,freqs,names
10610,4793,приложение_noun
15486,4358,я_npro
1384,4226,весь_adjf
4408,2783,игра_noun
8620,2744,очень_advb
1180,2477,быть_verb
11790,1874,реклама_noun
1982,1789,вы_npro
7627,1417,обновление_noun
7298,1413,нет_pred


In [211]:
frequent_words = word_freqs[:50]
freq_nouns = frequent_words[frequent_words.names.str.endswith("noun")]
freq_nouns

,freqs,names
10610,4793,приложение_noun
4408,2783,игра_noun
11790,1874,реклама_noun
7627,1417,обновление_noun
2867,1018,деньга_noun
1870,939,время_noun
4925,876,карта_noun
11333,870,раз_noun
11512,740,разработчик_noun
1646,717,водитель_noun


In [244]:
def strtonormalform(line, should_delete = {'PREP', 'CONJ', 'PRCL', 'INTJ'}):
    ans = ''
    words = re.findall(r"[\w]+[\w|-]*", line)
    for word in words:
        lexInfo = morph.parse(word)[0]
        lex = lexInfo.normal_form
        if (lexInfo.tag.POS not in should_delete) and (re.match(r"[-+]?\d+$", lex) is None) :#союзы, предлоги и т.п. не сохраняем
            ans = ans + ' ' + lex
            if (lexInfo.tag.POS):
                ans = ans + "_" + lexInfo.tag.POS

def tag_words_POS(words_list):
    new_list = []
    for word in words_list:
        lexInfo = morph.parse(word)[0]
        if (lexInfo.tag.POS):
            word = word + "_" + lexInfo.tag.POS
        new_list.append(word)
    return new_list

def tag_words_POS_dict(words_dict):
    new_dict = {}
    for word in words_dict.keys():
        lexInfo = morph.parse(word)[0]
        if (lexInfo.tag.POS):
            new_word = word + "_" + lexInfo.tag.POS
        new_dict.update({new_word : words_dict[word]})
    return new_dict

In [254]:
aspect_general = ["приложение", "игра", "программа"]
aspect_general = tag_words_POS(aspect_general)
aspect_general

['приложение_NOUN', 'игра_NOUN', 'программа_NOUN']

In [261]:
sent_words = {
    "ужасный" : -1,
    "интересный" : 1,
    "супер" : 1,
    "замечательный" : 1,
    "шедевр" : 1
}

sent_words = tag_words_POS_dict(sent_words)
sent_words

{'замечательный_ADJF': 1,
 'интересный_ADJF': 1,
 'супер_ADJF': 1,
 'ужасный_ADJF': -1,
 'шедевр_NOUN': 1}

In [256]:
test_df = reviews[:500]
test_df


,rating,content
0,1,описание_NOUN приложение_NOUN написать_VERB р...
1,1,сделать_VERB русский_ADJF язык_NOUN
2,2,зайти_VERB игра_NOUN поиграть_VERB ровно_ADVB...
4,5,быть_VERB любезный_ADJS возможность_NOUN доба...
6,4,русский_ADJF играть_INFN невозможно_ADVB выбо...
7,5,почему_ADVB игра_NOUN запускаться_VERB переки...
8,1,ждать_INFN заявить_PRTF описание_NOUN русский...
9,2,русский_ADJF язык_NOUN нет_PRED английский_AD...
10,5,товарищ_NOUN учить_VERB иностранный_ADJF язык...
12,1,заявить_PRTS русский_ADJF язык_NOUN заходить_...


In [275]:
df1 = pd.DataFrame(columns = test_df.columns)


for index, row in test_df.iterrows():
    row.content = row.content.split()
    row['asp_pos'] = []
    flag = False
    for word in aspect_general:
        if (word in row.content):
            flag = True
            row['asp_pos'] += [row.content.index(word)]
    if (flag):
        df1 = df1.append(row)
df1

,rating,content,asp_pos
0,1.0,"[описание_NOUN, приложение_NOUN, написать_VERB...","[1, 6]"
2,2.0,"[зайти_VERB, игра_NOUN, поиграть_VERB, ровно_A...",[1]
6,4.0,"[русский_ADJF, играть_INFN, невозможно_ADVB, в...",[5]
7,5.0,"[почему_ADVB, игра_NOUN, запускаться_VERB, пер...","[10, 1]"
9,2.0,"[русский_ADJF, язык_NOUN, нет_PRED, английский...",[15]
10,5.0,"[товарищ_NOUN, учить_VERB, иностранный_ADJF, я...",[4]
12,1.0,"[заявить_PRTS, русский_ADJF, язык_NOUN, заходи...",[4]
13,5.0,"[очень_ADVB, продумать_PRTF, интересный_ADJF, ...",[3]
14,3.0,"[игра_NOUN, нармуля_NOUN, разрабочик_NOUN, доб...",[0]
21,3.0,"[игра_NOUN, хороший_ADJF, хватать_VERB, русски...",[0]


AttributeError: 'list' object has no attribute 'shape'

<class 'int'>


In [318]:
df2 = pd.DataFrame(columns = df1.columns)

win_size = 2

for index, row in df1.iterrows():
    for word in sent_words:
        for pos in row['asp_pos']:
            min_pos = max(0, pos-win_size)
            list_len = len(row['content']) -1
            max_pos = min(pos+win_size, list_len)
            if (word in row['content'][min_pos:max_pos]):
                df2 = df2.append(row)

df2

,rating,content,asp_pos
13,5.0,"[очень_ADVB, продумать_PRTF, интересный_ADJF, ...",[3]
35,4.0,"[игра_NOUN, супер_ADJF, часто_ADVB, пропадать_...",[0]
76,5.0,"[добавить_VERB, русский_ADJF, язык_NOUN, игра_...",[3]
104,5.0,"[добавить_VERB, русский_ADJF, язык_NOUN, игра_...",[3]
128,5.0,"[игра_NOUN, шедевр_NOUN, добавить_VERB, русски...",[0]
151,3.0,"[задумка_NOUN, отличный_ADJF, игра_NOUN, интер...",[2]
170,3.0,"[весь_ADJF, супер_ADJF, игра_NOUN, прекрасный_...",[2]
179,3.0,"[игра_NOUN, супер_ADJF, играть_VERB, минута_NO...",[0]
342,2.0,"[игра_NOUN, интересный_ADJF, большой_ADJF, отс...",[0]
357,2.0,"[игра_NOUN, интересный_ADJF, большой_ADJF, отс...",[0]
